# Modelo de recomendación item-item

Se genera un modelo de ML de recomendacion de items por similitud del coseno es decir yo ingreso un articulo y el modelo busca articulos similares y nos los recomienda comparando que tan parecidos son.

El modelo de la similitud del coseno funciona comparando vectores, los cuales obtenemos de nuestros datasets, los vectores son comparados en un espacio en el que se evalua el valor del coseno comprendido entre ellos.

## Carga de archivos necesarios y transformaciones

In [87]:
import pandas as pd


In [88]:
 
df = pd.read_csv("games.csv")
df.head()

,publisher,genres,app_name,title,id,developer,release_year,price
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,2018,4.99
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,2018,4.99
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,2018,4.99
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,2018,4.99
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,2018,4.99


In [89]:
# Se requiere el id y filtrar por titulo en pasos posteriores por lo que debo obtenerlo

dfD = df.groupby('title')['id'].sum().reset_index()

In [90]:
dfD.head()

,title,id
0,! That Bastard Is Trying To Steal Our Gold !,1799760
1,"""Barely Attuned Magic Thingy"" Staff",1540815
2,"""Glow Ball"" - The billiard puzzle game",1553560
3,"""Just Another Day"" - Seduce Me Otome CD",909580
4,"""Lethargic Sentience"" Wand",1540820


El modelo no requiere de precio, año en que fue lanzado ni del id por lo que se quitan del dataframe.

In [91]:
df=df.drop(["price","id","release_year","app_name"], axis=1)
df.head()

,publisher,genres,title,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Kotoshiro
1,Kotoshiro,Casual,Lost Summoner Kitty,Kotoshiro
2,Kotoshiro,Indie,Lost Summoner Kitty,Kotoshiro
3,Kotoshiro,Simulation,Lost Summoner Kitty,Kotoshiro
4,Kotoshiro,Strategy,Lost Summoner Kitty,Kotoshiro


In [92]:
# se agrupan los generos en una sola fila.
genres_df = df.groupby('title')['genres'].agg(', '.join).reset_index()

In [93]:
genres_df.head()

,title,genres
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie"
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl..."
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy"
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation"
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl..."


In [94]:
# Se agrupan tambien el publisher por cada fila para eso se requiere antes cambiar el tipo de dato a str

df["publisher"] = df["publisher"].astype(str)

publisher_df = df.groupby('title')['publisher'].agg(', '.join).reset_index()

In [95]:
publisher_df.head()

,title,publisher
0,! That Bastard Is Trying To Steal Our Gold !,"WTFOMGames, WTFOMGames, WTFOMGames, WTFOMGames"
1,"""Barely Attuned Magic Thingy"" Staff","Sin dato, Sin dato, Sin dato, Sin dato, Sin dato"
2,"""Glow Ball"" - The billiard puzzle game","WTFOMGames, WTFOMGames, WTFOMGames, WTFOMGames"
3,"""Just Another Day"" - Seduce Me Otome CD","Sin dato, Sin dato"
4,"""Lethargic Sentience"" Wand","Sin dato, Sin dato, Sin dato, Sin dato, Sin dato"


In [96]:
# Se unen los dataframes por titulo
dfGames =  pd.merge(genres_df,publisher_df, on="title")
dfGames.head()

,title,genres,publisher
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie","WTFOMGames, WTFOMGames, WTFOMGames, WTFOMGames"
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl...","Sin dato, Sin dato, Sin dato, Sin dato, Sin dato"
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy","WTFOMGames, WTFOMGames, WTFOMGames, WTFOMGames"
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation","Sin dato, Sin dato"
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl...","Sin dato, Sin dato, Sin dato, Sin dato, Sin dato"


In [99]:
dfGames.drop(["publisher"], axis=1)

,title,genres
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie"
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl..."
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy"
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation"
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl..."
...,...,...
30050,（尘沙惑设定集）Lost in Secular Love - Concept Design ...,"Adventure, Casual, Indie, Simulation"
30051,４人打ちアクション麻雀 / ACTION MAHJONG,"Action, Casual, Indie, Early Access"
30052,＜/reality＞,"Adventure, Indie"
30053,＜/reality＞ Original Soundtrack,"Adventure, Indie"


In [101]:
dfModelo = dfGames

In [107]:
dfModelo.to_csv("dfModelo", index=False)

In [108]:
dfMod = pd.read_csv("dfModelo")

## Modelo de recomendacion item- item

En este apartado se genera el modelo de recomendacion item- item por la similitud del coseno.

Se utilizo la libreria scikit learn para este proceso

In [103]:
# Se importa vectorizer de sklearn

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 

In [104]:
cv = CountVectorizer(max_features = 5000, stop_words="english")

In [105]:
cv.fit_transform(dfMod["genres"]).toarray().shape

(30055, 34)

In [106]:
# Se generan los vectores a comparar 
vectores = cv.fit_transform(dfMod["genres"]).toarray()

In [15]:
# Se observan la forma de los vectores
vectores

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
# se llama la liberia de sklearn para la similitud del coseno
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# se aplica la similitud del coseno a nuestros vectores
similitud = cosine_similarity(vectores)

In [20]:
# Se obtiene el array de similitud
similitud[0]

array([1.        , 0.37796447, 0.5       , ..., 0.70710678, 0.70710678,
       0.70710678])

In [21]:
# Ordenamos la similitud entre más similar a menos similar tomando 5 valores
sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x:x[1])[1:6]

[(158, 1.0), (327, 1.0), (513, 1.0), (656, 1.0), (657, 1.0)]

In [36]:
# Se genera una funcion que te da la recomendación por titulo
def recomendacion(game):
    indice_juegos = dfMod[dfMod["title"]==game].index[0]
    distances = similitud[indice_juegos]
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_titles = [dfMod.iloc[i[0]]['title'] for i in lista_juegos]
    return recommended_titles

In [40]:
# Se prueba la función 
recomendacion('"Glow Ball" - The billiard puzzle game')

['Adorables',
 'Fish or Die',
 'LASER STRIKERS',
 'More Than Just Chess',
 'TREBUCHET']

## Generación de una tabla de resultados 

Debido a las limitaciones de render se decidio aplicar el modelo a cada titulo diferente para así obtener un dataframe con las recomendaciones del modelo.

In [41]:
# Se aplica la función al dataframe para obtener una nueva columna con las recomendaciones ya que es más facil de cargar
dfMod['recomendaciones'] = dfMod['title'].apply(recomendacion)

In [43]:
dfMod.head()

,title,genres,recomendaciones
0,! That Bastard Is Trying To Steal Our Gold !,"Action, Adventure, Casual, Indie","[36 Fragments of Midnight, A Goo Adventure, AN..."
1,"""Barely Attuned Magic Thingy"" Staff","Action, Free to Play, Indie, Massively Multipl...","[""Lethargic Sentience"" Wand, ""Precisely Calibr..."
2,"""Glow Ball"" - The billiard puzzle game","Casual, Indie, Sports, Strategy","[Adorables, Fish or Die, LASER STRIKERS, More ..."
3,"""Just Another Day"" - Seduce Me Otome CD","Casual, Simulation","[Agricultural Simulator 2013 - Steam Edition, ..."
4,"""Lethargic Sentience"" Wand","Action, Free to Play, Indie, Massively Multipl...","[""Lethargic Sentience"" Wand, ""Precisely Calibr..."


Al realizar la API se evidencia que debo trabajar con id por lo que se convinan dataframes para obtenerlo

In [45]:
MODELO = dfMod.drop(["genres"], axis=1)

In [47]:
MODELO.to_parquet("Modelo.parquet")

In [48]:
M1 = pd.read_parquet("data/Modelo.parquet")

In [54]:
MODELO = pd.merge(dfD,M1, on="title")

In [86]:
MODELO.head()

,id,recomendaciones
0,1799760,"[36 Fragments of Midnight, A Goo Adventure, AN..."
1,1540815,"[""Lethargic Sentience"" Wand, ""Precisely Calibr..."
2,1553560,"[Adorables, Fish or Die, LASER STRIKERS, More ..."
3,909580,"[Agricultural Simulator 2013 - Steam Edition, ..."
4,1540820,"[""Lethargic Sentience"" Wand, ""Precisely Calibr..."


In [57]:
#Se guarda en parquet para mayor eficiencia
MODELO.to_parquet("Modelo.parquet")

In [58]:
# Se genera una copia del dataframe para su restauración 
Modelo = MODELO

In [74]:
Modelo.head()

,id,recomendaciones
0,1799760,"[36 Fragments of Midnight, A Goo Adventure, AN..."
1,1540815,"[""Lethargic Sentience"" Wand, ""Precisely Calibr..."
2,1553560,"[Adorables, Fish or Die, LASER STRIKERS, More ..."
3,909580,"[Agricultural Simulator 2013 - Steam Edition, ..."
4,1540820,"[""Lethargic Sentience"" Wand, ""Precisely Calibr..."


In [75]:
Modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30055 entries, 0 to 30054
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               30055 non-null  int32 
 1   recomendaciones  30055 non-null  object
dtypes: int32(1), object(1)
memory usage: 352.3+ KB


Se hicieron pruebas con las siguientes funciones ya que corren en este entorno pero no en la API

In [76]:
# Esta función sirve en el notebook pero no en la API por lo que sr tuvo que cambiar
def recomendacion_juego(id_producto):
    recomendacion = Modelo[Modelo['id'] == id_producto]['recomendaciones'].iloc[0]
    
    return {
        'Segun el id que proporcionaste te podrian interesar estos juegos': recomendacion
    }

In [77]:
recomendacion_juego(1553560)

{'Segun el id que proporcionaste te podrian interesar estos juegos': array(['Adorables', 'Fish or Die', 'LASER STRIKERS',
        'More Than Just Chess', 'TREBUCHET'], dtype=object)}

In [82]:
# Esta funcion es compatible con la API por lo que fue la que se dejo 
def obtener_recomendaciones_dict_por_id(id_producto):
    recomendaciones = Modelo[Modelo['id'] == id_producto]['recomendaciones'].iloc[0]
    
    # Verificar si la lista de recomendaciones no está vacía
    if len(recomendaciones) > 0:
        recomendaciones_dict = {i + 1: juego for i, juego in enumerate(recomendaciones)}
        return recomendaciones_dict
    else:
        # Si la lista de recomendaciones está vacía, devolver un diccionario vacío
        return {}

In [84]:
recomendaciones_dict = obtener_recomendaciones_dict_por_id(1540815)
print(recomendaciones_dict)

{1: '"Lethargic Sentience" Wand', 2: '"Precisely Calibrated Stringstick" Bow', 3: '"Toy Knife" Dagger', 4: '"Unstable Anomaly" Sword', 5: 'Gangs of Space - Panther Pack'}
